# [Egg Dropping Problem](https://brilliant.org/wiki/egg-dropping/)
- You are given 
    + K: eggs 
    + N: a building with N floors from 1 to N.
        + Each move, you may take an egg and drop it from any floor X, X in range [0,N].
    + F: exists a floor F in range [0,N] that
        + Any egg dropped at a floor higher than F will break
        + Any egg dropped at or below floor F will not break.
- The goal is to know the value of F within minimum number of moves
    + And find the minimum number of moves in the worst case

## Infinitite of Eggs $(K = \infty)$
- Binary search: Number of worst case moves = $log_2(N)$
- Example N = 100, the worst case is F = 1, $log_2(100) = 7$
    + Drop at floor 100
    + Drop at floor 50
    + Drop at floor 25
    + Drop at floor 12
    + Drop at floor 6
    + Drop at floor 3
    + Drop at floor 1
    
## 1 Egg $(K = 1)$
- Linear Search: Number of worst case moves = $N$
- Example N = 100, the worst case is F = 100
    + Drop at floor 1
    + Drop at floor 2
    + Drop at floor 3
    + ...
    + Drop at floor 100

## 2 Eggs $(K = 2)$
- Divide and Conquer
- Assume we drop the first egg from floor X
    + If it breaks, we can use the 2nd egg and step through floor [1, X-1] one-by-one.
        + Number of move in the worst case: 1 + (X-1 -1 + 1) = X
    + If it doesn't break, We continue to use the 1st egg.
- The next floor should be from floor X + (X-1) to ensure the Number of move in the worst case is the same with the prior drop
    + If it breaks, we can use the 2nd egg and step through floor [X+1, 2X-2] one-by-one.
        + Number of move in the worst case: 1 + 1 + (2X-2 - (X+1) +1) = X
    + If it doesn't break, We continue to use the 1st egg.
- Deduce from that 
$$X + (X-1) + (X-2) + ... + 1 \geqslant N$$
$$=>\ \frac{X(X+1)}{2}\geqslant N$$

- Example N = 100, Number of worst case moves X = 14

| Drop 	| Floor 	|
|:----:	|:-----:	|
|  #1  	|   14  	|
|  #2  	|   27  	|
|  #3  	|   39  	|
|  #4  	|   50  	|
|  #5  	|   60  	|
|  #6  	|   69  	|
|  #7  	|   77  	|
|  #8  	|   84  	|
|  #9  	|   90  	|
|  #10 	|   95  	|
|  #11 	|   99  	|
|  #12 	|  100  	|

## [General case N floors, K eggs](https://leetcode.com/problems/super-egg-drop/)
- Examples

```
Input: K = 1, N = 2
Output: 2
Explanation: 
    Drop the egg from floor 1.  If it breaks, we know with certainty that F = 0.
    Otherwise, drop the egg from floor 2.  If it breaks, we know with certainty that F = 1.
    If it didn't break, then we know with certainty F = 2.
    Hence, we needed 2 moves in the worst case to know what F is with certainty.
    
Input: K = 2, N = 6
Output: 3

Input: K = 3, N = 14
Output: 4
```

#### Solution DP $O(KN^2)$
- K eggs, N floors, drop at floor X
    + f(K, N): worstcase moves with K eggs, N floors
    + f(K-1, X-1): the egg K break at floor X, continue with K-1 eggs, X-1 floors
    + f(K, N-X): the egg K survive at floor X, continue with K eggs, N-X floors
 
- $f(K,N) = \min\limits_{\forall X \in  [1,N]}( 1 + \max(f(K-1,X-1),\ f(K,N-X))$

    
```C++
class Solution {
public:
    const int inf = INT_MAX;
    vector<vector<int>> dp;

    int get(int K, int N) {
        if(N == 0) return 0;
        if(N == 1) return 1;
        if(K == 1) return N;
        if(dp[K][N] != inf) return dp[K][N];

        for(int X=1; X<=N; ++X) 
            dp[K][N] = min(dp[K][N], 1 + max(get(K-1, X-1), get(K, N-X)));
        
        return dp[K][N]; 
    }
    int superEggDrop(int K, int N) {
        dp.assign(K+1, vector<int>(N+1, inf));
        return get(K,N);
    }
};
```

#### Solution DP $O(KNlog(N))$
- Let
    + $f_1 = f(K-1,X-1)$
    + $f_2 = f(K,N-X)$
- Observations
    + $f_1$ increase when X increase
    + $f_2$ decrease when X increase
        + => $max(f_1, f_2) = f_1$ when $f_1 > f_2$
        + => $max(f_1, f_2) = f_2$ when $f_2 > f_1$
        + We can use binary search to find the optimum of $min(1 + max(f_1, f_2))$ (when $f_1 = f_2$) 

<img src="./img/9.png" alt="drawing" width="300"/>


```C++
class Solution {
public:
    const int inf = INT_MAX;
    vector<vector<int>> dp;

    int get(int K, int N) {
        if(N == 0) return 0;
        if(N == 1) return 1;
        if(K == 1) return N;
        if(dp[K][N] != inf) return dp[K][N];

        int l = 1;
        int r = N;
        int optimum = inf;
        while(l < r) {
            int X = l+(r-l)/2;

            int f1 = get(K-1, X-1);
            int f2 = get(K, N-X);
            optimum = 1 + max(f1, f2);

            if(f1 == f2) break;
            if(f1 < f2) l = X+1;
            else r = X;
        }
        
        return dp[K][N] = optimum; 
    }
    int superEggDrop(int K, int N) {
        dp.assign(K+1, vector<int>(N+1, inf));
        return get(K,N);
    }
};
```

#### Solution Math $O(Klog(N))$
- Suppose we try t times with K eggs
    + The result would be a sequence of fails/successes, len = t with at most K failed. Eg: t=8, 00101101, 00101010
        + The number of sequences with 1 failed throw is $C_t^1$
        + The number of sequences with 2 failed throw is $C_t^2$
        + ...
        + The number of sequences with K failed throw is $C_t^K$
    - Call g(t,K) is the total number of sequences: g(t,K) = $C_t^1 + C_t^2 + ... + C_t^K = \sum\limits_{1\leq x \leq K}C_t^x$
    - g(t,K) can be calculated in O(N) by the formula $C_t^x = \frac{t-x+1}{x}C_t^{x-1}$
- We want to try as less as possible:
    + Find the minimum t that $g(t,K) \geqslant N$  
    + We can use binary search to find t in range [1, N]

```C++
class Solution {
public:
    int N;

    int g(int t, int K) {
        int sum = 0;
        int C = 1;

        for(int x=1; x<=K; ++x) {
            C = (t-x+1)*C / x;
            sum += C;

            // g(t, K) too large, stop calculating
            if(sum >= N) break;
        }
        return sum;
    }
    int superEggDrop(int K, int N) {
        this->N = N;
        int l = 1;
        int r = N;
        while(l < r) {
            int t = l+(r-l)/2;
            if(g(t, K) < N) l = t + 1;
            else r = t;
        }
        return l;
    }
};
```